In [ ]:
import akshare as ak
import pandas as pd

In [ ]:
au9999 = ak.spot_hist_sge(symbol='Au99.99')
aum = ak.futures_hist_em(symbol="沪金主连", period="daily")

In [ ]:
df = pd.merge(au9999, aum, left_on='date', right_on='时间', how='left')
df = df.rename(columns = {'close': 'au9999', '收盘': 'aum'})
df = df[['date', 'au9999', 'aum']]
df['date'] = pd.to_datetime(df['date'])
df['premium'] = df['au9999'] - df['aum']
df['premium'].abs().describe()

In [ ]:
def calculate_drawdown_time(row, val):
     current_date = row['date']
     current_premium_abs = abs(row['premium'])
     if current_premium_abs <= 2.5:
         return pd.Timedelta(0)
     subsequent_data = df[df['date'] > current_date]
     drawdown_index = subsequent_data[subsequent_data['premium'].abs() <= val].first_valid_index()
     if drawdown_index is not None:
         drawdown_date = df.loc[drawdown_index, 'date']
         return drawdown_date - current_date
     return pd.NaT
sell_point = 2.5

df[f'down_{sell_point}'] = df.apply(calculate_drawdown_time, args=(sell_point,), axis=1)

In [ ]:
threshold = 5
df[(df['premium'] > threshold) | (df['premium'] < -1 * threshold)]